In [5]:
import numpy as np
import pandas as pd
import random
import pyesg

DEFAULT_CONFIG = {
    'yrs': 40,
    'freq': 12,
    'n_scen': 10,
    'age': 30,
    'sex': 'M',
    'smoker': 'NS',
    'ret_age': 65,
    'mi': 0.99,
    'fund_value': 100000,
    'ret_inc': 1000000,
    'salary': 100000,
    'salary_growth': 0.045,
    'invest_pct': 0.15,
    'inflation': 0.02,
    's_pct': 0.80,
    'pct_5': 0.55,
    's_pct_end': 0.2,
}

def age_series(age=30, freq=12, yrs=40, **kwargs):
    """Produces an array that denotes the age at each period"""
    age_array = np.ones(freq * yrs + 1) / freq
    age_array[0] = age
    age_array = np.cumsum(age_array)
    age_s = pd.Series(age_array, name="age")
    return age_s


def salary_series(
    salary=100000, salary_growth=0.04, freq=12, yrs=40, age=30, ret_age=65, **kwargs
):
    """Produces an array that determines the salary at each period."""
    income_period = ret_age - age
    salary_growth_per_period = (1 + salary_growth) ** (1 / freq) - 1
    salary_growth_array = np.ones(freq * yrs + 1) * (1 + salary_growth_per_period)
    salary_growth_array[0] = salary
    salary_array = np.cumprod(salary_growth_array)
    salary_array[(income_period * freq) :] = 0
    salary_s = pd.Series(salary_array, name="salary")
    return salary_s


def invest_series(
    salary_s=None,
    invest_pct=0.15,
    freq=12,
    ret_inc=100000,
    age=30,
    ret_age=65,
    **kwargs
):
    """
    Produces an array that determines the amount to be invested or withdrawn at each period. For a
    given period, if age < retirement age, investments will be made. Once retirement is reached,
    investments will cease and withdrawals will begin.
    """
    income_period = ret_age - age
    if salary_s is None:
        salary_s = salary_series(**kwargs)
    invest_pct_per_period = invest_pct / freq
    invest = salary_s * invest_pct_per_period
    invest[income_period * freq :] = -1 * ret_inc / freq
    invest.name = "invest"
    return invest


# simulate mortality for n scenarios starting at retirement
# mi is mortality improvement
def mortality_sim(
    age=32, ret_age=65, sex="F", smoker="NS", n_scen=1000, mi=0.99, **kwargs
):
    """
    Runs a monte carlo simulation on n scenarios to determine age at death. Each item in the
    resulting array represent the age at death for that scenario.
    """
    mort_df = pd.read_csv("MortalityTables/cso2017.csv")
    mort_df = mort_df.set_index(["Sex", "SmokingStatus", "IssueAge"])
    row = mort_df.loc[(sex, smoker, ret_age)]
    arr = np.empty(n_scen)

    for n in range(n_scen):
        d = 0
        j = ret_age
        i = row[d] / 1000 * (mi ** (j - age))
        while random.random() > i:
            j += 1
            d += 1
            if d < len(row):
                i = row[d] / 1000 * (mi ** (j - age))
            else:
                break
        arr[n] = j

    df = pd.DataFrame(arr, columns=["age_at_death"])

    return df

def asset_mix(
    start_pct=0.90,  # pct invested in equities at the start of the projection
    pct_5=0.55,  # pct invested in equities 5 yrs out from retirement
    end_pct=0.40,  # pct invested in equities at retirement
    age=30,  # age at the start of the projection
    ret_age=65,  # retirement age and the age at which the end asset_mix pct is reached
    freq=12,  # the number of periods in a year
    yrs=40,  # length of the projection in yrs
    **kwargs
):
    """
    Produces a 2 x n multidimensional array that defines the asset mix between stocks and bonds at
    a given period. the sum of the two arrays is always equal to 1.0.
    """
    # income_period represents the number of income earning yrs
    income_period = ret_age - age

    # segment_one_length represents the number of periods at the initial constant asset_mix
    segment_one_length = 0 if income_period <= 30 else (income_period - 30) * freq

    # segment_four_length represents the number of periods at the ending constant asset_mix
    segment_four_length = (
        0 if yrs <= income_period else (yrs - income_period) * freq + 1
    )

    # calculate segment_two_length, start pct and end pct
    # segment two represents the period from 30 to 5 yrs from retirement
    segment_two_slope = (pct_5 - start_pct) / (25 * freq)
    segment_two_end_pct = pct_5 - segment_two_slope
    if income_period <= 5:
        segment_two_length = 0
        segment_two_start_pct = 0
    elif income_period >= 30:
        segment_two_length = 25 * freq
        segment_two_start_pct = start_pct
    else:
        segment_two_length = (income_period - 5) * freq
        segment_two_start_pct = pct_5 + ((income_period - 5) / 25 * (start_pct - pct_5))

    # calculate segment_three_length, start pct and end pct
    # segment three represents the period from 5 to 0 yrs from retirement
    segment_three_slope = (end_pct - pct_5) / (5 * freq)
    segment_three_end_pct = end_pct - segment_three_slope
    if income_period >= 5:
        segment_three_length = 5 * freq
        segment_three_start_pct = pct_5
    else:
        segment_three_length = income_period * freq
        segment_three_start_pct = end_pct + (income_period / 5 * (pct_5 - end_pct))

    # setup asset_mix arrays
    segment_one_array = np.full(segment_one_length, start_pct)
    segment_two_array = np.linspace(
        segment_two_start_pct, segment_two_end_pct, segment_two_length
    )
    segment_three_array = np.linspace(
        segment_three_start_pct, segment_three_end_pct, segment_three_length
    )
    segment_four_array = np.full(segment_four_length, end_pct)

    # concatenate the arrays into one single array
    stock_asset_mix_array = np.concatenate(
        (segment_one_array, segment_two_array, segment_three_array, segment_four_array)
    )
    bond_asset_mix_array = 1 - stock_asset_mix_array

    # add the stock and bond asset_mix arrays into a single multidimensional array
    result_array = np.column_stack((stock_asset_mix_array, bond_asset_mix_array))
    df = pd.DataFrame(result_array, columns=["stock", "bond"])

    return df

def fund_projection(**kwargs):
    config = DEFAULT_CONFIG.copy()
    config.update(kwargs)
    
    s_pct = config['s_pct']
    freq = config['freq']
    yrs = config['yrs']
    fund_value = config['fund_value']

    b_pct = 1 - s_pct
    n_steps = freq * yrs
    dt = 1/freq
    x0 = 100
    n_scen = yrs * freq

    # instantiate a new model with the required parameters
    stock_model = pyesg.GeometricBrownianMotion(mu=0.10, sigma=0.15)
    bond_model = pyesg.GeometricBrownianMotion(mu=0.05, sigma=0.05)

    # run model for both equities and bonds
    s_model_results = stock_model.scenarios(x0, dt, n_scen, n_steps)
    b_model_results = bond_model.scenarios(x0, dt, n_scen, n_steps)

    # create stock and bond index return arrays. 
    stock_return = s_model_results[:, 1:] / s_model_results[:, :-1]
    bond_return = b_model_results[:, 1:] / b_model_results[:, :-1]

    # set beginning of fund array to starting investment value
    stock_array = np.insert(stock_return, 0, fund_value * s_pct, axis=1)
    bond_array = np.insert(bond_return, 0, fund_value * b_pct, axis=1)

    # the last return value is not used so we add a 1 to the end to return the array to its original length
    ones_to_append = np.ones((stock_return.shape[0], 1), dtype=int)
    stock_return = np.append(stock_return, ones_to_append, axis=1)
    bond_return = np.append(bond_return, ones_to_append, axis=1)

    # create pandas series for various calcs
    invest_s = invest_series(**config)
    asset_mix_s = asset_mix(**config)            

    # this is where the magic happens
    # calc the fund value at each point in time, credit interest, add/withdraw from fund
    for s, b, inv, alloc_s, alloc_b in zip(stock_array, bond_array, invest_s, asset_mix_s['stock'], asset_mix_s['bond']):
        for i in range(1, len(s)):
            s[i] = s[i-1] * s[i]                            # stock fund @t = stock fund @t-1 * stock return
            b[i] = b[i-1] * b[i]                            # bond fund @t = bond fund @t-1 * bond return
            total_fund = s[i] + b[i] + inv[i-1]/freq        # total fund = stock + bond fund +/- investment
            s[i] = total_fund * alloc_s                     # reallocate fund value to stock fund
            b[i] = total_fund * alloc_b                     # reallocate fund value to bond fund

    total_fund = stock_array + bond_array
    df = pd.DataFrame(total_fund.T)

    return df


In [6]:
df = fund_projection()

In [7]:
print(df)

              0             1             2             3             4    \
0    1.000000e+05  1.000000e+05  1.000000e+05  1.000000e+05  1.000000e+05   
1    9.974475e+04  1.001327e+05  9.844763e+04  1.016296e+05  9.895684e+04   
2    1.095391e+05  1.004584e+05  9.292885e+04  1.021892e+05  1.026622e+05   
3    1.176147e+05  9.980306e+04  9.315548e+04  1.067111e+05  1.111871e+05   
4    1.261119e+05  1.036172e+05  1.015407e+05  1.100945e+05  1.087496e+05   
..            ...           ...           ...           ...           ...   
476  1.832316e+06  1.373446e+07  1.054317e+06  4.843939e+06  5.231618e+06   
477  1.856599e+06  1.357323e+07  1.001342e+06  4.742943e+06  5.715096e+06   
478  1.855132e+06  1.447155e+07  1.031885e+06  4.840930e+06  6.013170e+06   
479  1.866275e+06  1.491993e+07  1.055043e+06  4.866687e+06  5.868283e+06   
480  1.905277e+06  1.551482e+07  1.100208e+06  4.796023e+06  5.954635e+06   

              5             6             7             8             9    

In [4]:
salary = 200000
config = {
    'salary':salary,
    'salary_growth':0.04,
    'freq':12,
    'yrs':40,
    'age':30,
    'ret_age':65,
    'test':2
}
salary_s = salary_series(**config)
print(salary_s)

0      200000.000000
1      200654.747956
2      201311.639387
3      201970.681310
4      202631.880764
           ...      
476         0.000000
477         0.000000
478         0.000000
479         0.000000
480         0.000000
Name: salary, Length: 481, dtype: float64


In [12]:
from asset_mix import asset_mix

asset_mix_df = asset_mix()
age_s = age_series()
invest_s = invest_series()
salary_s = salary_series()

df = pd.concat([age_s, asset_mix_df, invest_s, salary_s], axis=1)
print(df)


           age  equity  bond       invest         salary
0    30.000000     0.9   0.1  1250.000000  100000.000000
1    30.083333     0.9   0.1  1254.092175  100327.373978
2    30.166667     0.9   0.1  1258.197746  100655.819694
3    30.250000     0.9   0.1  1262.316758  100985.340655
4    30.333333     0.9   0.1  1266.449255  101315.940382
..         ...     ...   ...          ...            ...
476  69.666667     0.4   0.6 -8333.333333       0.000000
477  69.750000     0.4   0.6 -8333.333333       0.000000
478  69.833333     0.4   0.6 -8333.333333       0.000000
479  69.916667     0.4   0.6 -8333.333333       0.000000
480  70.000000     0.4   0.6 -8333.333333       0.000000

[481 rows x 5 columns]
